In [1]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [27]:
import os
import zipfile

from pathlib import Path
image_path = Path("data/version3")


In [28]:
# Setup directories
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir


PosixPath('data/version3/train')

In [29]:
# Setup ImageNet normalization levels (turns all images into similar distribution as ImageNet)
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])

# # Create transform pipeline manually
# manual_transforms = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     normalize
# ])

# print(f"Manually created transforms: {manual_transforms}")

image_size=(224,224)
transforms_train = transforms.Compose([transforms.ToPILImage(),
                                       transforms.Resize(image_size),
                                       transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.1),
                                       transforms.RandomRotation(degrees=30),
                                       transforms.ToTensor()
                                       ])

transforms_test = transforms.Compose([transforms.ToPILImage(),
                                      transforms.Resize(image_size),
                                      transforms.ToTensor()
                                       ])


# # Setup pretrained weights (plenty of these available in torchvision.models)
# weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# # Get transforms from weights (these are the transforms that were used to obtain the weights)
# automatic_transforms = weights.transforms() 
# print(f"Automatically created transforms: {automatic_transforms}")


# Create data loaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    #transform=automatic_transforms, # use automatic created transforms
    transforms_train=transforms_train, # use automatic created transforms
    transforms_test=transforms_test, # use automatic created transforms
    batch_size=32
)

train_dataloader, test_dataloader, class_names

FileNotFoundError: Found no valid file for the classes .ipynb_checkpoints. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp